# Re-entrenamos el modelo medium de YoloV5 con el dataset entero de Kaggle

Añadimos acceso a la carpeta de los dataset del Drive del grupo de proyectos

Importante estar ya logeados en la cuenta del grupo, da problemas si estamos en nuestra cuenta personal y queremos montar el drive de la cuenta del grupo.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos las bibliotecas necesarias

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
# Para windows con la CPU:
!pip3 install torch torchvision torchaudio 

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

# Para windows con la CPU:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio

# Es posible que la mayoría de dependencias ya esten instaladas ^^

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!git clone https://github.com/ultralytics/yolov5
# Se deberia haber creado una carpeta en nuestro area de trabajo
# Si ya lo teniamos saldra el siguiente error =>fatal: destination path 'yolov5' already exists and is not an empty directory.

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [5]:
%cd /content/yolov5
!git pull
%cd ../

/content/yolov5
Already up to date.
/content


In [6]:
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2 #Para abrir webcam
# cv2.imshow() is disabled in Colab, because it causes Jupyter sessionsto crash, consider using cv2_imshow 
from google.colab.patches import cv2_imshow 
import sys # to access the system

# Esta línea de código establece una variable de entorno llamada KMP_DUPLICATE_LIB_OK con el valor "True", lo que significa que permite la carga de bibliotecas duplicadas en el entorno de Python. Esto es útil para algunas bibliotecas de Machine Learning que usan OpenMP para acelerar el cómputo.
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

***
En la siguiente celda se escogen 100 imagenes para hacer el re-entreno.
 
Además esas imagenes se suben a unas carpetas que creamos en el directorio base de colabs.
***

In [8]:
import os
import random
import shutil

# Creamos los directorios necesarios
os.makedirs('/content/data/train/images', exist_ok=True)
os.makedirs('/content/data/train/labels', exist_ok=True)
os.makedirs('/content/data/val/images', exist_ok=True)
os.makedirs('/content/data/val/labels', exist_ok=True)
os.makedirs('/content/data/test/images', exist_ok=True)
os.makedirs('/content/data/test/labels', exist_ok=True)

dest_train_Image = "/content/data/train/images/"
dest_train_Label = "/content/data/train/labels/"
dest_val_Image = "/content/data/val/images/"
dest_val_Label = "/content/data/val/labels/"
dest_test_Image = "/content/data/test/images/"
dest_test_Label = "/content/data/test/labels/"


with os.scandir('/content/drive/MyDrive/IA/Datasets/Señales_Plastico_Augmentation/train/images') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_train_Image)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Señales_Plastico_Augmentation/train/labels') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_train_Label)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Señales_Plastico_Augmentation/val/images') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_val_Image)

with os.scandir('/content/drive/MyDrive/IA/Datasets/Señales_Plastico_Augmentation/val/labels') as entries:
  for entry in entries:
    shutil.copy2(entry.path, dest_val_Label)



***
**`Entrenar un modelo desde cero`**
***

Ahora tenemos que crear el archivo dataset.yaml, que contiene las categorias de nuestros pesos y las rutas de nuestras imagenes de entrenamiento, test y validación.

In [9]:
f= open("yolov5/data/dataset_senalesTraficoCarmen_xl.yaml","w+")

#f.write("path: ../data \ntrain: train/images/ \nval: val/images \ntest: test/images \n\nnc: 4 \n\nnames: ['prohibitory', 'danger', 'mandatory', 'other']")
f.write("path: ../data \ntrain: train/images/ \nval: val/images \n\nnc: 4 \n\nnames: ['prohibitory', 'danger', 'mandatory', 'other']")

f.close()

In [10]:
shutil.copy2("/content/yolov5/requirements.txt", "/content/requirements.txt")
!cd yolov5 && python train.py --img 640 --batch 8 --epochs 300 --data dataset_senalesTraficoCarmen_xl.yaml --weights yolov5x.pt --project ../drive/MyDrive/IA/Pesos_yoloV5/Dataset_SenalesTrafico_Carmen_xl --save-period 20 

train: weights=yolov5x.pt, cfg=, data=dataset_senalesTraficoCarmen_xl.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../drive/MyDrive/IA/Pesos_yoloV5/Dataset_SenalesTrafico_Carmen_xl, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=20, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-153-gff6a9ac Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=

In [11]:
 #!cd yolov5 && python train.py --resume /content/drive/MyDrive/IA/Pesos_yoloV5/Dataset_SenalesTrafico_Carmen/exp/weights/last.pt

***
**`Cargamos nuestro modelo`**
***

In [12]:
# modelo = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/IA/Pesos_yoloV5/Dataset_Completo/exp/weights/best.pt', force_reload=True)

In [13]:
# import os

# with os.scandir('/content/drive/MyDrive/IA/Datasets/Señales_Plastico/') as entries:
#   for src_img in entries:
#     # print(src_img.path)
#     results = modelo(src_img.path)
#     plt.imshow(np.squeeze(results.render()))
#     plt.show()